In [5]:
import cnn_builder as cbuild
import copy
import glob
import importlib
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import niftiutils.masks as masks
import numpy as np
import os
import shutil
from os.path import *
import brats.train_isensee2017
%matplotlib inline
from scipy.ndimage.morphology import binary_closing, binary_opening, binary_dilation
from skimage.morphology import ball, label

In [89]:
importlib.reload(hf)

<module 'niftiutils.helper_fxns' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\helper_fxns.py'>

In [2]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

## CT, Lipiodol seg

In [82]:
importlib.reload(masks)

<module 'niftiutils.masks' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\masks.py'>

In [11]:
def seg_liver(img, save_folder, ct_dims):
    #B3 = ball(3)
    #B3 = B3[:,:,[0,2,3,4,6]]
    B3 = ball(4)
    B3 = B3[:,:,[1,3,5,6,8]]
    
    liver_mask = copy.deepcopy(img)
    liver_mask = liver_mask > 50
    liver_mask = binary_opening(binary_closing(liver_mask, B3), B3)
    
    masks.save_mask(liver_mask, join(save_folder, "liver_pred"), ct_dims, save_mesh=True)

In [12]:
info = {}

for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        
    seg_liver(ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims)
    break

In [88]:
def seg_lipiodol(img, save_folder, ct_dims):
    low_mask = copy.deepcopy(img)
    low_mask = low_mask > 100
    low_mask = binary_opening(binary_closing(low_mask, structure=np.ones((2,2,1))), structure=np.ones((2,2,1)))
    mid_mask = copy.deepcopy(img)
    mid_mask = mid_mask > 150
    mid_mask = binary_closing(mid_mask)
    high_mask = copy.deepcopy(img)
    high_mask = high_mask > 200
    
    masks.save_mask(low_mask, join(save_folder, "low_lipiodol"), ct_dims, save_mesh=True)
    masks.save_mask(mid_mask, join(save_folder, "mid_lipiodol"), ct_dims, save_mesh=True)
    masks.save_mask(high_mask, join(save_folder, "high_lipiodol"), ct_dims, save_mesh=True)

In [102]:
info = {}

for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)
        
    seg_lipiodol((mask > 0) * ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims, mask)
    break

## Non-target Lipiodol seg

In [132]:
def seg_target_lipiodol(img, save_folder, ct_dims, num_tumors=1):
    mid_mask = copy.deepcopy(img)
    mid_mask = mid_mask > 150
    mid_mask = binary_closing(mid_mask)
    
    mask1 = copy.deepcopy(img)
    mask1 = mask1 > 150
    B2 = ball(2)
    B2 = B2[:,:,[0,2,4]]
    mask1 = binary_dilation(mask1, np.ones((3,3,1)))
    
    if num_tumors == 1:
        tumor_labels, num_labels = label(mask1, return_num=True)
        label_sizes = [np.sum(tumor_labels == label_id) for label_id in range(1,num_labels+1)]
        biggest_label = label_sizes.index(max(label_sizes))+1
        mask1[tumor_labels != biggest_label] = 0
    
    mask1 = binary_opening(binary_closing(mask1, structure=B2, iterations=2), structure=B2, iterations=2)
    
    if num_tumors == 1:
        tumor_labels, num_labels = label(mask1, return_num=True)
        label_sizes = [np.sum(tumor_labels == label_id) for label_id in range(1,num_labels+1)]
        biggest_label = label_sizes.index(max(label_sizes))+1
        mask1[tumor_labels != biggest_label] = 0
    
    target_mask = mask1 * mid_mask
    nontarget_mask = (1-mask1) * mid_mask
    
    masks.save_mask(mask1, join(save_folder, "tumor_pred"), ct_dims, save_mesh=True)
    masks.save_mask(target_mask, join(save_folder, "target_lip"), ct_dims, save_mesh=True)
    masks.save_mask(nontarget_mask, join(save_folder, "nontarget_lip"), ct_dims, save_mesh=True)

In [ ]:
info = {}

for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):
    if dirname(seg) in info.keys():
        (ct_img, ct_dims, mask) = info[dirname(seg)]
    else:
        ct_img, ct_dims = hf.dcm_load(join(dirname(seg), "CT24h"), True, True)
        mask, _ = masks.get_mask(seg, ct_dims, ct_img.shape)
        
    seg_target_lipiodol((mask > 0) * ct_img, dirname(seg), ct_dims)
    info[dirname(seg)] = (ct_img, ct_dims, mask)
    #break

### other CT

In [73]:
src_dir = r"C:\Users\Clinton\Downloads\24h CT tumor segmentions (Milena's data)"

In [75]:
for seg in glob.glob(join(src_dir,"tumor 24h_*")):
    patient_id = seg[seg.rfind('_Pat')+4:seg.rfind('.')]
    if len(patient_id) == 1:
        patient_id = "0"+patient_id
    dest = join(target_dir, patient_id)
    #if not exists(join(dest,basename(seg))):
    shutil.move(seg, dest)
    #print(seg, dest)

In [27]:
ct_img = tr.apply_window(ct_img)
ct_img -= np.amin(ct_img)

In [6]:
for ctfolder in glob.glob(join(target_dir,"*","CT24h")):
    os.rename(ctfolder, join(dirname(ctfolder), "CT24h"))

In [ ]:
for seg in glob.glob(join(target_dir,"*","*CT*")):
    contents = os.listdir(seg)
    filterstr = ' 1 5 '
    target_folder = [x for x in contents if filterstr in x]
    others = [x for x in contents if filterstr not in x]
    if len(target_folder) == 1:
        for o in others:
            shutil.rmtree(join(seg, o))

# BLMRI

In [12]:
path_dict = {}
patient_id = '04'
for path in glob.glob(join(target_dir, patient_id, "MRI-BL","*","*")):
    header = hf.dcm_load_header(path)
    if len(header) > 0:
        series_descript = header[0][('0008', '103e')].value
        path_dict[path] = series_descript
        series_descript = series_descript.replace("/","-")
        os.rename(path, join(target_dir,patient_id, "MRI-BL", series_descript))

In [ ]:
dcm_paths = [x for x in glob.glob(join(target_dir, "0Ready", "*", "MRI-BL","*")) if x.find('.')==-1]
for dcm_path in dcm_paths:
    if not exists(splitext(dcm_path)[0] + ".nii.gz"):
        try:
            hf.dcm_load(dcm_path, save_path=splitext(dcm_path)[0] + ".nii.gz")
        except:
            print(dcm_path)

In [7]:
art_paths = [x for x in glob.glob(join(target_dir, "0Ready", "*", "MRI-BL","*.nii.gz")) if '20s' in x or 'Ph1-Ax' in x]
for path in art_paths:
    os.rename(path, join(dirname(path), "t1-art.nii.gz"))

# Masks

In [299]:
maskpaths = [x for x in glob.glob(join(target_dir, "0Ready", "*", "*.ids")) if 'liver bl' in x]
#art_paths = glob.glob(join(target_dir, "0Ready", "*", "MRI-BL", "t1-art.nii.gz"))
for path in maskpaths:
    if exists(join(dirname(path), "MRI-BL", "truth.nii.gz")):
        continue
    art_path = join(dirname(path), "MRI-BL", "t1-art.nii.gz")
    img, img_dims = hf.nii_load(art_path, True, True)
    mask, dims = masks.get_mask(path, img_dims=img_dims, img_shape=img.shape)
    hf.save_nii(mask, join(dirname(path), "MRI-BL", "truth.nii.gz"), dims)

[32.192, 93.187, 31.421] [3.183, 3.183, 3.183]


In [283]:
importlib.reload(masks)

<module 'niftiutils.masks' from 'c:\\users\\clinton\\documents\\niftiutils\\niftiutils\\masks.py'>

In [63]:
for patient in os.listdir(img_dir):
    if not exists(join(target_dir, patient[:2], "CT")):
        paths = [x for x in os.listdir(os.path.join(img_dir, patient)) if x.endswith('CT') or x[:-2].endswith('CT')]
        for path in paths:
            subpaths = [x for x in os.listdir(os.path.join(img_dir, patient, path)) if x.find('.')==-1]
            if len(subpaths) > 1:
                print(patient, path)
            elif len(subpaths) == 1:
                if not exists(join(target_dir, patient[:2], "CT")):
                    shutil.copytree(join(img_dir, patient, path, subpaths[0]), join(target_dir, patient[:2], "CT"))
                else:
                    shutil.copytree(join(img_dir, patient, path, subpaths[0]), join(target_dir, patient[:2], "CT2"))
        #print(os.path.join(img_dir, patient, ct24_path))

In [49]:
for patient in os.listdir(img_dir):
    blmr_paths = [x for x in os.listdir(os.path.join(img_dir, patient)) if x.endswith('MRI baseline')]
    #os.makedirs(join(target_dir, patient))
    for blmr_path in blmr_paths:
        shutil.copytree(join(img_dir, patient, blmr_path), join(target_dir, patient, "MRI-BL"))
    #print(os.path.join(img_dir, patient, ct24_path))

# UNet

In [3]:
import numpy as np
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import *

In [4]:
from keras.optimizers import Adam
#from unet3d.metrics import dice_coefficient_loss, get_label_dice_coefficient_function, dice_coefficient
K.set_image_data_format("channels_first")
from keras.layers.merge import concatenate

In [25]:
importlib.reload(cbuild)
model = cbuild.build_ct_unet()

In [ ]:
for seg in glob.glob(join(target_dir,"*","wholeliver_24hCT*.ids")):

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 128, 128, 128 0                                            
__________________________________________________________________________________________________
reshape_12 (Reshape)            (None, 1, 128, 128,  0           input_15[0][0]                   
__________________________________________________________________________________________________
conv3d_60 (Conv3D)              (None, 16, 128, 128, 304         reshape_12[0][0]                 
__________________________________________________________________________________________________
dropout_55 (Dropout)            (None, 16, 128, 128, 0           conv3d_60[0][0]                  
__________________________________________________________________________________________________
batch_norm

In [ ]:
model.fit()